In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import re

def freqGuesser(radius, thickness, material=None):
    """
    freqGuesser estimates the mechanical mode frequencies of disk resonators.
    
    Parameters:
    radius (float): radius of disk resonator - must be supplied
    thickness (float): thickness of disk resonator - must be supplied
    material (str, optional): material of the disk resonator ('silicon' or 'silica')
    
    Returns:
    pd.DataFrame: A DataFrame with estimated frequencies
    
    """
    if radius is None or thickness is None:
        raise ValueError("Dimensional properties (radius and thickness) must be supplied")
    
    # Default material properties for silica
    material_properties = {
        'youngs': 73E9,
        'pratio': 0.16,
        'density': 2330
    }
    
    # Override properties if material is provided
    if material:
        if 'silicon' in material:
            material_properties = {
                'youngs': 163E9,
                'pratio': 0.16,
                'density': 2202
            }
        elif 'silica' in material:
            material_properties = {
                'youngs': 73E9,
                'pratio': 0.23,
                'density': 2330
            }

    Bessel = np.array([
        [0, 2.94440720000000, 6.17369440000000, 9.34575880000000],
        [0, 4.50270220000000, 7.71528180000000, 10.8860160000000],
        [2.39935160000000, 5.94130180000000, 9.17529020000000, 12.3624480000000],
        [3.63525420000000, 7.30172260000000, 10.5778520000000, 13.7906720000000],
        [4.79804320000000, 8.60952400000000, 11.9373000000000, 15.1808020000000],
        [5.92590360000000, 9.87941140000000, 13.2629040000000, 16.5398140000000],
        [7.03237440000000, 11.1205060000000, 14.5610400000000, 17.8727420000000],
        [8.12407860000000, 12.3388580000000, 15.8363180000000, 19.1833740000000],
        [9.20481140000000, 13.5386640000000, 17.0921640000000, 20.4746140000000],
        [10.2769660000000, 14.7229880000000, 18.3312180000000, 21.7487720000000],
        [11.3421540000000, 15.8941120000000, 19.5555420000000, 23.0076840000000],
        [12.4015200000000, 17.0537880000000, 20.7667940000000, 24.2528660000000],
        [13.4558740000000, 18.2033940000000, 21.9663060000000, 25.4855660000000],
        [14.5058580000000, 19.3440240000000, 23.1551980000000, 26.7068340000000],
        [15.5519420000000, 20.4765680000000, 24.3343820000000, 27.9175620000000],
        [16.5945000000000, 21.6017520000000, 25.5046300000000, 29.1184880000000],
        [17.6338300000000, 22.7201760000000, 26.6665940000000, 30.3102800000000],
        [18.6701600000000, 23.8323600000000, 27.8208360000000, 31.4934980000000],
        [19.7036880000000, 24.9387100000000, 28.9678460000000, 32.6686300000000]
    ])

    Kfactor = np.sqrt(material_properties['youngs'] / 12 / (1 - material_properties['pratio']**2) / material_properties['density']) * thickness / (2 * np.pi * radius**2)
    Frequencies = Bessel**2 * Kfactor
    
    Families = pd.DataFrame({
        'Butterfly(p=0)': Frequencies[:, 0],
        'First Family(p=1)': Frequencies[:, 1],
        'Second Family(p=2)': Frequencies[:, 2],
        'Third Family(p=3)': Frequencies[:, 3]
    })

    return Families







# List of measured frequencies
MeasuredF = [2518.662, 2518.704, 2518.712, 2518.734, 2518.87, 2518.9, 2518.93, 2519.356, 5768.001, 5768.08, 5768.144, 5768.144, 5768.236, 10020.34, 10020.43, 10020.43, 10020.51, 10020.65, 10026.47, 10052.15, 15237.25, 15237.455, 15237.53, 15237.72, 15238.573, 15308.52, 15308.555, 15308.605, 15309.265, 15322.016, 21380.75, 21381.112, 21381.122, 21381.14, 21484.593, 22999.124, 22999.174, 22999.268, 22999.28, 22999.336, 22999.578, 23102.832, 28417.752, 28418.18, 28418.196, 28418.252, 28420.965]

# Use freqGuesser to generate frequencies
radius = 75.6E-3 / 2
thickness = 2.3995E-3
generated_frequencies = freqGuesser(radius, thickness)

# Use DBSCAN to cluster the measured frequencies
db = DBSCAN(eps=10, min_samples=1).fit(np.array(MeasuredF).reshape(-1, 1))
labels = db.labels_

# Find closest frequency in each cluster from the generated frequencies
clustered_frequencies = []
for label in np.unique(labels):
    cluster_indices = np.where(labels == label)[0]
    cluster_frequencies = np.array(MeasuredF)[cluster_indices]
    cluster_center = np.mean(cluster_frequencies)
    
    # Find the closest frequency in the generated dataframe
    closest_frequency_index = (generated_frequencies - cluster_center).abs().stack().idxmin()
    family_name = closest_frequency_index[1]
    match_frequency = generated_frequencies.loc[closest_frequency_index[0], family_name]
    clustered_frequencies.append((cluster_center, family_name, match_frequency,closest_frequency_index[0]))


# Create a new dataframe with the family title for each matched frequency
matched_df = pd.DataFrame(clustered_frequencies, columns=['Measured Frequency', 'Family', 'Modelled Frequency','n'])



# Display the result
# Overwrite values in the 'Family' column at specific indices
matched_df.loc[[5, 6], 'Family'] = 'First Family (p=1)'

matched_df['m'] = matched_df['Family'].apply(lambda x: re.search(r'p=(\d+)', x).group(1) if re.search(r'p=(\d+)', x) else None)


# Display the DataFrame to check the changes
reorder = ['Measured Frequency','Modelled Frequency','Family','m','n']
print(matched_df[reorder])

matched_df.to_csv('labelled_frequencies.csv', index=False)